In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler

plt.rcParams["figure.figsize"] = (10,8) 

In [2]:
df = pd.read_pickle("../datos/possum_normalizado.pkl")

In [3]:
df.sample(10)

,age,hdlngth,skullw,taill,footlgth,earconch,eye,chest,belly,hdlngth_esta,case,site,Pop,sex,totlngth
2,1.140433,0.392875,1.005785,1.020671,1.586856,0.921668,0.434173,1.473667,0.514241,0.392875,3,1,Vic,f,95.5
95,0.087726,0.617839,-0.382018,0.507870,-0.447933,-0.105333,-1.000805,-0.982445,0.150424,0.617839,96,7,other,m,83.0
19,0.087726,0.617839,-0.188371,0.507870,1.221051,1.043930,0.434173,0.000000,1.241874,0.617839,20,1,Vic,f,89.0
12,0.614080,0.702201,0.973511,-0.517732,0.580893,0.408167,0.721168,0.000000,-0.213392,0.702201,13,1,Vic,m,89.5
86,-0.964982,1.658297,1.231707,2.302673,0.740932,-0.325405,-0.044153,-0.491222,1.241874,1.658297,87,7,other,m,93.0
51,1.140433,1.405212,1.328530,1.533472,-0.127854,-0.936716,0.721168,0.736833,-0.031484,1.405212,52,3,other,m,93.5
46,-0.964982,-0.703823,-0.672489,0.251470,-0.562247,-0.643287,-0.044153,-0.982445,0.150424,-0.703823,47,3,other,m,89.0
22,0.087726,-0.028932,-0.252920,-0.517732,0.992423,1.264002,0.338508,0.491222,0.878057,-0.028932,23,1,Vic,f,89.0
35,1.666787,0.196032,0.779864,-1.030532,1.335365,0.946120,-0.139818,-0.736833,1.241874,0.196032,36,2,Vic,m,88.0
42,-0.964982,-0.731943,-0.446567,-2.568935,0.809521,0.310358,-1.574796,0.982445,-0.577209,-0.731943,43,2,Vic,f,81.0


In [4]:
lm = ols('totlngth ~  age  + skullw + taill + footlgth + earconch + eye + chest + belly + hdlngth_esta', data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
age,1.0,126.785330,126.785330,24.370614,3.580173e-06
skullw,1.0,354.197246,354.197246,68.083621,1.153203e-12
taill,1.0,354.295634,354.295634,68.102533,1.146895e-12
footlgth,1.0,346.807161,346.807161,66.663102,1.744427e-12
earconch,1.0,1.480052,1.480052,0.284495,5.950705e-01
eye,1.0,5.745371,5.745371,1.104372,2.960913e-01
chest,1.0,29.307372,29.307372,5.633449,1.972404e-02
belly,1.0,1.609418,1.609418,0.309361,5.794374e-01
hdlngth_esta,1.0,67.670181,67.670181,13.007529,5.063671e-04
Residual,91.0,473.417086,5.202386,NaN,NaN


In [5]:
lm = ols('totlngth ~  skullw + taill + footlgth + chest + hdlngth_esta', data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
skullw,1.0,531.048553,531.048553,99.213397,1.630847e-16
taill,1.0,376.082903,376.082903,70.261867,4.107604e-13
footlgth,1.0,326.634174,326.634174,61.023584,6.725816e-12
chest,1.0,50.615649,50.615649,9.456293,2.733616e-03
hdlngth_esta,1.0,93.365925,93.365925,17.443133,6.471988e-05
Residual,97.0,519.201146,5.352589,NaN,NaN


### En el momento de elegir las variables predictoras, hicimos una tabla de correlación y vimos que era muy alta entre skull y hdlngth, por lo que decidimos quedarnos sólo con una de ellas(hdlngth).
### tras el anova, parece que tiene más efecto en nuestra variable respuesta, la predictora skull.
### A tener en cuenta a la hora de realizar nuestro modelo. 

In [6]:
lm = ols('totlngth ~  age ', data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
age,1.0,120.544439,120.544439,7.266885,0.008242
Residual,100.0,1658.818698,16.588187,NaN,NaN


In [7]:
lm = ols('totlngth ~  site + Pop + sex + age', data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
site,6.0,812.524176,135.420696,14.140683,2.219322e-11
Pop,1.0,7.106079,7.106079,0.742020,3.912302e-01
sex,1.0,6.436047,6.436047,0.672055,4.144323e-01
age,1.0,63.845982,63.845982,6.666823,1.138393e-02
Residual,93.0,890.630606,9.576673,NaN,NaN


### Podemos descartar Pop y sex como variables que inlfuyan en nuestra variable respuesta. En cambio la edad (age) y el sitio de muestreo (site) sí que son variables a tener en cuenta. 
### Tiene sentido lógico que la edad influye, teneindo en cuenta el crecimiento natural de los animales... pero nos sorprende que el sitio dónde crecen sea inlfuyente. Una variable a estudiar y tener en cuenta. 

### Todas nuestras variables afectan a la variable respuesta, excepto tres: earconch, eye, belly, cuyo PR (p-valor) es superior a 0,05.
### Según F las variables que mejor capacidad para explicar nuestra variable respuesta son taill, skull y footlgth en ese orden. 
#### F = test que se utiliza para evaluar la capacidad explicativa que tiene la variable predictora sobre la variación de la variable respuesta. Es decir, pretende determinar si de entre todos los valores de la variable predictora, al menos una tiene capacidad de explicar una parte significativa de la variación de la variable respuesta.)


In [8]:
lm = ols('totlngth ~  site + age', data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
site,6.0,812.524176,135.420696,14.172989,1.960292e-11
age,1.0,68.683651,68.683651,7.188359,8.666759e-03
Residual,94.0,898.155311,9.554844,NaN,NaN


In [9]:
lm = ols('totlngth ~  site', data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
site,6.0,907.588591,151.264765,14.581728,8.452489e-12
Residual,97.0,1006.237563,10.373583,NaN,NaN


In [10]:
df["age_cat"]= df["age"].astype("object")

In [11]:
lm = ols('totlngth ~  site + age_cat', data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
site,6.0,812.524176,135.420696,15.553833,4.811315e-12
age_cat,8.0,209.366452,26.170806,3.005865,5.023765e-03
Residual,87.0,757.472510,8.706581,NaN,NaN


### COCLUSIONES: en nuestro modelo deberíamos invcluir las variables más significativas que son: site ( como variable categórica), age (que la procesamos tanto como numérica, cómo categórica)
### interpretar conclusiones!!! ¿Qué hacemos con age??